In [1]:
from _keys import db_user, db_password, db_name, db_host, db_port
import psycopg2
import time
import requests

In [2]:
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
cur = conn.cursor()

In [3]:
api = "https://api.pushshift.io/reddit/search/submission?author="
#4/1/2015 00:00:00
after_time = 1427860800
#1/1/2018 00:00:00
final_time = 1514782800
#response limit
response_limit = 25

In [4]:
sql = """select norm_user_name from public.norm_users"""

In [5]:
# Get all sus_user_names and save to list
names = []
cur.execute(sql)
for row in cur:
    names.append(row[0])

In [6]:
sql_insert = """INSERT INTO public.norm_user_posts (author, author_flair_text, created_utm, domain, id, over_18, permalink, subreddit_id, title, url, post_score, subreddit, top_lvl_comments) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [7]:
for name in names:
    counter = after_time
    user_post_count = 0
    request = requests.get(f'{api}{name}&after={counter}&before={final_time}&size={response_limit}')
    scanned_url = f'{api}{name}&after={counter}&before={final_time}&size={response_limit}'
    try:
        response = request.json()
        submissions = response['data']
        response_length = len(submissions)
        while counter < final_time and user_post_count < 99 and response_length > 0:
            try:
                counter = submissions[-1]['created_utc']
                for submission in submissions:
                    try:
                        cur.execute(sql_insert, (submission['author'], submission['author_flair_text'], submission['created_utc'], submission['domain'], submission['id'], submission['over_18'], submission['permalink'], submission['subreddit_id'], submission['title'], submission['url'], submission['score'], submission['subreddit'], submission['num_comments']))
                        conn.commit()
                        user_post_count += 1
                    except Exception as e:
                        # print(f'submission error ---->{e}')
                        next
            except Exception as e:
                # print(f'user error ---->{e}')
                time.sleep(1)
                next
    except Exception as e:
        # print(f'query error ---->{e}, {request.status_code}, retry after ')
        time.sleep(1)
        next